<a href="https://colab.research.google.com/github/Vlasovets/Deep_learning_course_assistantship/blob/master/layout_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x # Oleg's code

`%tensorflow_version` only switches the major version: `1.x` or `2.x`.
You set: `1.x # Oleg's code`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [33]:
import os
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.layers import Input, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.python.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

# Some GPUs require setting the `allow_growth` setting.
# Comment out this code is you don't have a GPU card.
import tensorflow.compat.v1 as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.Session(config=config)

In [0]:
!unzip layouts-easy.zip #Oleg unzips the folder

In [36]:
IMAGE_SIZE = (200, 200)
BATCH_SIZE = 64
NUM_EPOCHS = 100

dataset_dir = 'layouts-easy'
trainer_dir = dataset_dir + '/train'

# Specify manually the class labels, otherwise Keras will assign alphanumeric values
# as directories are listed (in no particular order).
dirname, class_labels, _ = next(os.walk(trainer_dir))
print('Class labels: {}'.format(class_labels))

Class labels: ['Good', 'Bad']


In [0]:
# Apply some data augmentation techniques.
image_data = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2
)

In [38]:
# Use generators since we might not be able to fit all the images into memory.
train_generator = image_data.flow_from_directory(
    trainer_dir,
    subset='training',
    color_mode='grayscale',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    classes=class_labels
)

Found 320 images belonging to 2 classes.


In [39]:
valid_generator = image_data.flow_from_directory(
    trainer_dir,
    subset='validation',
    color_mode='grayscale',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    classes=class_labels
)


Found 80 images belonging to 2 classes.


In [40]:
# Notice that images are grayscaled, therefore we have to set the number of channels to 1.
# Image shape is `(width, height, num_channels)` in TensorFlow.
in_shape = IMAGE_SIZE + (1,)

model = Sequential()
model.add(Conv2D(3, (3,3), activation='relu', input_shape=in_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 3)       30        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 3)         0         
_________________________________________________________________
dropout (Dropout)            (None, 99, 99, 3)         0         
_________________________________________________________________
flatten (Flatten)            (None, 29403)             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 29404     
Total params: 29,434
Trainable params: 29,434
Non-trainable params: 0
_________

In [0]:
# Setup some useful callbacks.
visualizer = TensorBoard(log_dir='/tmp/layouts')
checkpoint = ModelCheckpoint('{}-best.h5'.format(dataset_dir), monitor='val_acc', mode='max', save_best_only=True)
earlystops = EarlyStopping(patience=10)

In [42]:
# Train the model.
model.fit_generator(
    train_generator,
    epochs=NUM_EPOCHS,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // BATCH_SIZE,
    callbacks=[visualizer, checkpoint, earlystops]
)

Epoch 1/100
5/5 [==============================] - 5s 1s/step - loss: 345.5931 - acc: 0.5281 - val_loss: 299.5660 - val_acc: 0.4688
Epoch 2/100
5/5 [==============================] - 3s 627ms/step - loss: 187.8827 - acc: 0.5219 - val_loss: 188.2558 - val_acc: 0.5312
Epoch 3/100
5/5 [==============================] - 3s 618ms/step - loss: 138.6527 - acc: 0.4969 - val_loss: 44.1428 - val_acc: 0.5156
Epoch 4/100
5/5 [==============================] - 3s 622ms/step - loss: 74.9140 - acc: 0.5375 - val_loss: 31.7611 - val_acc: 0.6250
Epoch 5/100
5/5 [==============================] - 3s 618ms/step - loss: 50.1347 - acc: 0.5906 - val_loss: 42.2773 - val_acc: 0.5469
Epoch 6/100
5/5 [==============================] - 3s 623ms/step - loss: 36.3287 - acc: 0.6531 - val_loss: 14.3849 - val_acc: 0.7500
Epoch 7/100
5/5 [==============================] - 3s 608ms/step - loss: 29.3279 - acc: 0.6469 - val_loss: 29.1910 - val_acc: 0.6406
Epoch 8/100
5/5 [==============================] - 3s 600ms/step - 

In [43]:
# Report score from last epoch.
loss, acc = model.evaluate_generator(valid_generator)
print('Accuracy: {:.2f}%'.format(acc*100))

# Finally save the model.
model.save('{}.h5'.format(dataset_dir))

Accuracy: 83.75%
